In [1]:
import numpy as np
import pandas as pd

## Datasets:

MovieLens + IMDb/Rotten Tomatoes

---

### Description

---

This dataset is an extension of MovieLens10M dataset, published by GroupLeans 
research group.

http://www.grouplens.org 

It links the movies of MovieLens dataset with their corresponding web pages at 
Internet Movie Database (IMDb) and Rotten Tomatoes movie review systems.

http://www.imdb.com 

http://www.rottentomatoes.com 

From the original dataset, only those users with both rating and tagging information 
have been mantained.

---

### Data statistics

---
<table>
<tr style="border-bottom: 2pt solid black;"><th>count</th><th>name</th></tr>
<tr><td>2113</td><td>users</td> </tr>
<tr style="border-bottom: 2pt solid black;"><td>10197</td><td>movies</td></tr>

<tr><td>20</td><td>movie genres</td></tr>
<tr><td>20809</td><td>movie genre assignments</td> </tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 2.040 genres per movie</td></tr>

<tr><td>4060</td><td>directors</td></tr>
<tr><td>95321</td><td>actors</td></tr>
<tr><td></td><td>avg. 22.778 actors per movie</td></tr>
<tr style="border-bottom: 2pt solid black;"><td>72</td><td>countries</td></tr>

<tr><td>10197</td><td>country assignments</td></tr>
<tr><td></td><td>avg. 1.000 countries per movie</td></tr>
<tr><td>47899</td><td>location assignments</td></tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 5.350 locations per movie</td></tr>

<tr><td>13222</td><td>tags</td></tr>
<tr><td>47957</td><td>tag assignments (tas), i.e. tuples [user, tag, movie]</td></tr>
<tr><td></td><td>avg. 22.696 tas per user</td></tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 8.117 tas per movie</td></tr>

<tr><td>855598</td><td>ratings</td></tr>
<tr><td></td><td>avg. 404.921 ratings per user</td></tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 84.637 ratings per movie</td></tr>

</table>

\begin{array}
-R & = & R^u \cup R_{unknown} \\
R^u & = & R^u_{train} \cup R^u_{test}
\end{array}

## Metrics:

\begin{array}
-RMSE & =  &\sqrt{\frac{1}{|R_{test}|}\sum_{(u, i) \in R_{test}} (r_{ui} - \widehat{r}_{ui})^2} \\
MAP & = & \frac{\sum_{u \in U} AP^u}{|U|},
\end{array}
where $AP^u = \frac{1}{|Rel^u|} \sum_{(u, i) \in Rel^u} \frac{1}{k_i^u}$

In [8]:
data = pd.read_table('data/user_ratedmovies-timestamps.dat', encoding='cp1251', index_col=['userID', 'movieID'])
data.head(10)

rating      timestamp
userID movieID                       
75     3           1.0  1162160236000
       32          4.5  1162160624000
       110         4.0  1162161008000
       160         2.0  1162160212000
       163         4.0  1162160970000
       165         4.5  1162160715000
       173         3.5  1162160257000
       296         5.0  1162160689000
       353         3.5  1162160220000
       420         2.0  1162160202000